<a href="https://colab.research.google.com/github/bhstoller/multi-agent-customer-service/blob/main/agent_to_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/bhstoller/multi-agent-customer-service.git

Cloning into 'multi-agent-customer-service'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 76 (delta 31), reused 55 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 36.10 KiB | 9.03 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [2]:
!pip install uv -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 95.3 MB/s eta 0:00:00


In [3]:
# Install required packages
%pip install --upgrade -q google-genai google-adk==1.9.0 a2a-sdk==0.3.0 python-dotenv aiohttp uvicorn requests mermaid-python nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
import sys

from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver


class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver


patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module  # type: ignore

In [5]:
import sys

from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver
import asyncio
import logging
import os
import sys
import threading
import time

from typing import Any

import httpx
import nest_asyncio
import uvicorn

from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from dotenv import load_dotenv
from google.adk.a2a.executor.a2a_agent_executor import (
    A2aAgentExecutor,
    A2aAgentExecutorConfig,
)
from google.adk.agents import Agent, SequentialAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset
from google.adk.tools.mcp_tool.mcp_toolset import StreamableHTTPConnectionParams


/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [6]:
# Set Google Cloud Configuration
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'app-ai-a5'
os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'

load_dotenv()
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("a5-key")

print('Environment variables configured:')
print(f'GOOGLE_GENAI_USE_VERTEXAI: {os.environ["GOOGLE_GENAI_USE_VERTEXAI"]}')
print(f'GOOGLE_CLOUD_PROJECT: {os.environ["GOOGLE_CLOUD_PROJECT"]}')
print(f'GOOGLE_CLOUD_LOCATION: {os.environ["GOOGLE_CLOUD_LOCATION"]}')
print("GOOGLE_API_KEY loaded =", os.environ["GOOGLE_API_KEY"] is not None)

Environment variables configured:
GOOGLE_GENAI_USE_VERTEXAI: FALSE
GOOGLE_CLOUD_PROJECT: app-ai-a5
GOOGLE_CLOUD_LOCATION: us-central1
GOOGLE_API_KEY loaded = True


In [7]:
if 'google.colab' in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=os.environ['GOOGLE_CLOUD_PROJECT'])

In [8]:
# Setup logging
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
)

# **Add Agents**

In [9]:
MCP_SERVER_URL = userdata.get('MCP_SERVER_URL')
print(MCP_SERVER_URL)

https://polar-nonsolubly-madden.ngrok-free.dev/mcp


## Add Customer Agent

In [10]:
customer_data_agent = Agent(
    model="gemini-2.0-flash",
    name="customer_data_agent",
    tools=[ MCPToolset(
        connection_params=StreamableHTTPConnectionParams(
            url=MCP_SERVER_URL
            )
        )
    ],
    instruction="""
    You are the Customer Data Agent.

    Your role is to interface with the MCP server and perform EXACT data operations.

    REQUIRED RULES:
    - ALWAYS call an MCP tool for any operation involving customer data.
    - NEVER invent fields, values, or IDs.
    - ALWAYS return valid JSON.
    - If a customer does not exist, return:
      {"error": "Customer ID not found", "customer_id": <id>}
    - For successful lookups, return:
      {"customer": { ...mcp result... }}

    AVAILABLE OPERATIONS:
    - get_customer(customer_id)
    - list_customers(status, limit)
    - update_customer(customer_id, data)
    - create_ticket(customer_id, issue, priority)
    - get_customer_history(customer_id)

    ADDITIONAL RULES:
    - When updating a customer, preserve fields not being modified.
    - When creating a ticket, always include created_at returned by MCP.
    - When listing customers, return array of customer objects.

    DO NOT return natural language. ONLY JSON.
    """
)
print("Customer Data Agent created!")

Customer Data Agent created!


In [11]:
customer_data_agent_card = AgentCard(
    name='Customer Data Agent',
    url='http://127.0.0.1:10020',
    description='Fetches and updates customer data using MCP tools',
    version='1.0',
    capabilities=AgentCapabilities(streaming=False),
    default_input_modes=['text/plain'],
    default_output_modes=['application/json'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='customer_data_access',
            name='Customer Data Access',
            description='Retrieve and update customer records',
            tags=['customer', 'data', 'database', 'lookup'],
            examples=[
                'Get customer 42',
                'Update customer 5 email',
                'Show all customers',
            ],
        )
    ],
)


In [12]:
remote_customer_data_agent = RemoteA2aAgent(
    name='customer_data_agent',
    description='Remote wrapper for Customer Data Agent',
    agent_card=f'http://127.0.0.1:10020{AGENT_CARD_WELL_KNOWN_PATH}',  # ← FIXED
)

/tmp/ipython-input-2030816957.py:1: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  remote_customer_data_agent = RemoteA2aAgent(


## Specialist Agent

In [13]:
support_agent = Agent(
    model="gemini-2.0-flash-exp",
    name="support_agent",
    instruction="""
    You are the Support Agent.
    RULES:
    - DO NOT infer or create customer attributes (address, tier, status, balance, etc.)
    - DO NOT guess missing information.
    - ONLY use customer data exactly as provided by the Router.
    - If customer-specific data is required to answer, respond ONLY with:

    {
      "needs_customer_data": true,
      "reason": "<why>",
      "requested_fields": ["email", "tickets", "status", ...]
    }

    When the Router provides customer data:
    - Use only those fields.
    - Provide a natural-language support answer.
    - Do not return JSON unless explicitly requested by the Router.

    Your job: troubleshooting, escalation, and answering support questions.
    """,
)

In [14]:
support_agent_card = AgentCard(
    name='Support Agent',
    url='http://127.0.0.1:10021',   # FIXED
    description='Handles general support questions and escalates to customer data agent when needed',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='customer_support',
            name='Customer Support',
            description='Provides general support responses',
            tags=['support', 'help', 'troubleshooting'],
            examples=[
                'How do I reset my password?',
                'What is your refund policy?',
                'I need help with my account.',
            ],
        )
    ],
)


In [15]:
remote_support_agent = RemoteA2aAgent(
    name='support_agent',
    description='Remote wrapper for Support Agent',
    agent_card=f'http://127.0.0.1:10021{AGENT_CARD_WELL_KNOWN_PATH}',  # FIXED
)

/tmp/ipython-input-332038127.py:1: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  remote_support_agent = RemoteA2aAgent(


## Router Agent

In [16]:
router_agent = SequentialAgent(
    name='trend_analysis_host',
    instruction="""
    You are the Router (Orchestrator) Agent.

    Your job is to coordinate two remote A2A agents:

    1. Support Agent at http://localhost:10021
    2. Customer Data Agent at http://127.0.0.1:10020

    You DO NOT call them directly as tools.
    Instead you send A2A messages by asking the host runtime to call:

        CALL_SUPPORT_AGENT("<message>")
        CALL_CUSTOMER_DATA_AGENT("<message>")

    Your responsibilities:

    1. Analyze the user query
    2. Determine what information is needed
    3. Call the correct remote agents using these function names
    4. Combine their outputs
    5. Return FINAL JSON ONLY.

    VALID OUTPUT FORMAT:

    {
      "query": "...",
      "customer": <customer data or null>,
      "support_response": <string or null>,
      "tickets": <ticket list or null>,
      "updates": <update information or null>
    }

    RULES:
    - Never invent customer data. Always request it.
    - Detect support escalation signals:
      {"needs_customer_data": true, "query": "..."}
    - For multi-step tasks, perform them step-by-step.
    - For missing customer: return {"error": "Customer not found"}.
    """,
    sub_agents=[customer_data_agent, support_agent],
)


ValidationError: 1 validation error for SequentialAgent
instruction
  Extra inputs are not permitted [type=extra_forbidden, input_value='\n    You are the Router...omer not found"}.\n    ', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

In [ ]:
router_agent_card = AgentCard(
    name="Router Agent",
    url="http://localhost:10040",
    description="orchestrates customer data agent and support agent, routes to support and customer-data agents, and composes the final answer.",
    version="1.0",
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=["text/plain"],
    default_output_modes=["application/json"],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id="router_orchestration",
            name="Router / Orchestrator",
            description="Routes user queries to sub-agents using A2A calls and coordinates outputs.",
            tags=["router", "orchestrator", "workflow", "a2a"],
            examples=[
                "Help me access my account",
                "Look up customer 12",
                "Update the customer's email",
                "The customer cannot log in",
            ]
        )
    ]
)

# **Running**

In [ ]:
def create_agent_a2a_server(agent, agent_card):
    """Create an A2A server for any ADK agent.

    Args:
        agent: The ADK agent instance
        agent_card: The ADK agent card

    Returns:
        A2AStarletteApplication instance
    """
    runner = Runner(
        app_name=agent.name,
        agent=agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),
    )

    config = A2aAgentExecutorConfig()
    executor = A2aAgentExecutor(runner=runner, config=config)

    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )

    # Create A2A application
    return A2AStarletteApplication(
        agent_card=agent_card, http_handler=request_handler
    )

In [ ]:
# Apply nest_asyncio
nest_asyncio.apply()

# Store server tasks
server_tasks: list[asyncio.Task] = []


async def run_agent_server(agent, agent_card, port) -> None:
    """Run a single agent server."""
    app = create_agent_a2a_server(agent, agent_card)

    config = uvicorn.Config(
        app.build(),
        host='127.0.0.1',
        port=port,
        log_level='warning',
        loop='none',  # Important: let uvicorn use the current loop
    )

    server = uvicorn.Server(config)
    await server.serve()


async def start_all_servers() -> None:
    """Start all servers in the same event loop."""
    tasks = [
        asyncio.create_task(
            run_agent_server(customer_data_agent, customer_data_agent_card, 10020)
        ),
        asyncio.create_task(
            run_agent_server(support_agent, support_agent_card, 10021)
        ),
        asyncio.create_task(
            run_agent_server(router_agent, router_agent_card, 10040)
        ),
    ]

    await asyncio.sleep(2)

    print('All agent servers started!')
    print('   - Customer Data Agent: http://0.0.0.0:10020')
    print('   - Support Agent:       http://0.0.0.0:10021')
    print('   - Router Agent:        http://0.0.0.0:10022')

    try:
        # await asyncio.gather(*tasks)
        await asyncio.Event().wait()
    except KeyboardInterrupt:
        print('Shutting down servers...')


def run_servers_in_background() -> None:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_all_servers())


server_thread = threading.Thread(target=run_servers_in_background, daemon=True)
server_thread.start()

time.sleep(3)


In [ ]:
print('Customer Data Agent Card:')
print(customer_data_agent_card)
print('Support Agent Card:')
print(support_agent_card)
print('Host Agent Card:')
print(router_agent_card)

In [ ]:
class A2ASimpleClient:
    """A2A Simple to call A2A servers."""

    def __init__(self, default_timeout: float = 240.0):
        self._agent_info_cache: dict[
            str, dict[str, Any] | None
        ] = {}  # Cache for agent metadata
        self.default_timeout = default_timeout

    async def create_task(self, agent_url: str, message: str) -> str:
        """Send a message following the official A2A SDK pattern."""
        # Configure httpx client with timeout
        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0,
            read=self.default_timeout,
            write=10.0,
            pool=5.0,
        )

        async with httpx.AsyncClient(timeout=timeout_config) as httpx_client:
            # Check if we have cached agent card data
            if (
                agent_url in self._agent_info_cache
                and self._agent_info_cache[agent_url] is not None
            ):
                agent_card_data = self._agent_info_cache[agent_url]
            else:
                # Fetch the agent card
                agent_card_response = await httpx_client.get(
                    f'{agent_url}{AGENT_CARD_WELL_KNOWN_PATH}'
                )
                agent_card_data = self._agent_info_cache[agent_url] = (
                    agent_card_response.json()
                )

            # Create AgentCard from data
            agent_card = AgentCard(**agent_card_data)

            # Create A2A client with the agent card
            config = ClientConfig(
                httpx_client=httpx_client,
                supported_transports=[
                    TransportProtocol.jsonrpc,
                    TransportProtocol.http_json,
                ],
                use_client_preference=True,
            )

            factory = ClientFactory(config)
            client = factory.create(agent_card)

            # Create the message object
            message_obj = create_text_message_object(content=message)

            # Send the message and collect responses
            responses = []
            async for response in client.send_message(message_obj):
                responses.append(response)

            # The response is a tuple - get the first element (Task object)
            if (
                responses
                and isinstance(responses[0], tuple)
                and len(responses[0]) > 0
            ):
                task = responses[0][0]  # First element of the tuple

                # Extract text: task.artifacts[0].parts[0].root.text
                try:
                    return task.artifacts[0].parts[0].root.text
                except (AttributeError, IndexError):
                    return str(task)

            return 'No response received'

In [ ]:
a2a_client = A2ASimpleClient()

In [ ]:
async def test_customer_data():
    response = await a2a_client.create_task(
        'http://127.0.0.1:10020',  # correct
        'Look up customer with id 1'
    )
    print(response)

asyncio.run(test_customer_data())

In [ ]:
async def test_support_agent():
    response = await a2a_client.create_task(
        'http://localhost:10021',  # correct
        'A customer says they cannot log in. What should they do?'
    )
    print("Support Agent Response:\n", response)

asyncio.run(test_support_agent())

In [ ]:
async def test_router_agent():
    response = await a2a_client.create_task(
        'http://localhost:10040',
        'Update customer 1 email to new-email@example.com'
    )
    print("Router Agent Response:\n", response)

asyncio.run(test_router_agent())

In [ ]:
async def test_full_system():
    response = await a2a_client.create_task(
        'http://localhost:10040',   # ROUTER ONLY
        'I need help with my account, customer ID 1'
    )
    print("Full System Response:\n", response)

asyncio.run(test_full_system())